Mostly based on this tutorial: https://python.langchain.com/v0.2/docs/tutorials/sql_qa/

In [ ]:
from operator import itemgetter

from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_community.llms.ollama import Ollama
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

llm = Ollama(model="mistral")
db = SQLDatabase.from_uri("duckdb:///dbt/jaffle_shop.duckdb")

In [ ]:
chain_input = {"question": "List all customer first names beginning with M"}

write_query_prompt = PromptTemplate(
    input_variables=["input", "table_info"],
    partial_variables={"top_k": "5"},
    template="""You are a DuckDB expert. Given an input question, first create a syntactically correct DuckDB query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per DuckDB. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use today() function to get the current date, if the question involves "today". No clarifications. No Markdown formatting. No preamble.

Use the following format:
Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}""",
)
write_query = create_sql_query_chain(llm, db, prompt=write_query_prompt)
# write_query.get_prompts()
write_query.invoke(chain_input)

In [ ]:
execute_query = QuerySQLDataBaseTool(db=db)

write_execute_chain = write_query | execute_query

write_execute_chain.invoke(chain_input)

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

write_execute_answer_chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

write_execute_answer_chain.invoke(chain_input)